In [ ]:
import time
from threading import Lock, Thread

orders = []
orders_lock = Lock()


def add_order(order):
    with orders_lock:
        orders.append(order)
        print(f"Добавлен заказ: {order}")


def process_order(worker_name):
    while True:
        with orders_lock:
            if orders:
                order = orders.pop(0)
                print(f"{worker_name} обрабатывает заказ: {order}")
            else:
                break
        if order:
            make_pizza(worker_name, order["pizza_name"], or-der["cooking_time"])
        else:
            time.sleep(1)  # Ожидание новых заказов


def make_pizza(worker_name, pizza_name, cooking_time):
    print(f"{worker_name} начинает готовить пиццу {pizza_name}...")
    time.sleep(cooking_time)
    print(f"{worker_name} закончил готовить пиццу {pizza_name}!")


if __name__ == '__main__':
    # Добавление заказов
    add_order({"pizza_name": "Маргарита", "cooking_time": 10})
    add_order({"pizza_name": "Пепперони", "cooking_time": 12})
    add_order({"pizza_name": "Гавайская", "cooking_time": 14})
    add_order({"pizza_name": "Четыре сыра", "cooking_time": 16})

    # Запуск потоков для обработки заказов
    worker1 = Thread(target=process_order, args=("Работник 1",))
    worker2 = Thread(target=process_order, args=("Работник 2",))
    worker3 = Thread(target=process_order, args=("Работник 3",))

    worker1.start()
    worker2.start()
    worker3.start()

    worker1.join()
    worker2.join()
    worker3.join()

    print("Все заказы обработаны.")


In [ ]:
from threading import Thread, Condition
import time


class Pizzeria:
    def __init__(self) -> None:
        self.pizza_ready: bool = False
        self.condition = Condition()

    def cook_pizza(self, order: str) -> None:
        with self.condition:
            print(f"Повар начал готовить пиццу {order}")
            time.sleep(2)  # Имитация процесса готовки
            print(f"Пицца {order} готова")
            self.pizza_ready = True
            self.condition.notify_all()

    def get_pizza(self, order: str) -> None:
        with self.condition:
            while not self.pizza_ready:
                print(f"Клиент ожидает пиццу {order}")
                self.condition.wait()
            print(f"Клиент получил пиццу {order}")
            self.pizza_ready = False


def serve_pizzas(pizzeria: Pizzeria, orders: list[str]) -> None:
    for order in orders:
        cook_thread = Thread(target=pizzeria.cook_pizza, args=(order,))
        client_thread = Thread(target=pizzeria.get_pizza, args=(order,))

        cook_thread.start()
        client_thread.start()

        cook_thread.join()
        client_thread.join()


if __name__ == '__main__':
    pizzeria = Pizzeria()
    orders = [
        'Маргарита',
        'Пепперони',
        'Гавайская',
        'Четыре сыра',
    ]
    serve_pizzas(pizzeria, orders)


In [ ]:
from threading import Thread, Semaphore
import time
import random


class Pizzeria:
    def __init__(self, num_tables: int) -> None:
        self.semaphore = Semaphore(value=num_tables)

    def eat_pizza(self, client_id: int) -> None:
        print(f"Клиент {client_id} хочет есть пиццу")
        self.semaphore.acquire()

        print(f"Клиент {client_id} садится за стол")
        eating_time = random.randint(1, 5)
        time.sleep(eating_time)  # Имитация процесса еды

        print(f"Клиент {client_id} освобождает стол")
        self.semaphore.release()


def serve_clients(pizzeria: Pizzeria, client_ids: list[int]) -> None:
    threads = []

    for client_id in client_ids:
        thread = Thread(target=pizzeria.eat_pizza, args=(client_id,))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()


if __name__ == '__main__':
    pizzeria = Pizzeria(num_tables=3)
    client_ids = list(range(1, 7))
    serve_clients(pizzeria, client_ids)


In [ ]:
from threading import Thread, Event
import time


def cook_pizza(event: Event) -> None:
    print("Повар начал готовить пиццу")
    time.sleep(5)  # Имитация процесса готовки
    print("Пицца готова")
    event.set()


def wait_for_pizza(event: Event) -> None:
    print("Клиент ждет пиццу")
    event.wait()
    print("Клиент получил пиццу")


if __name__ == '__main__':
    event = Event()

    cook_thread = Thread(target=cook_pizza, args=(event,))
    wait_thread = Thread(target=wait_for_pizza, args=(event,))

    cook_thread.start()
    wait_thread.start()

    cook_thread.join()
    wait_thread.join()
